# Model transforming a sequence of digits into a number
- input: sequence of three digits
- output: number in range(0,DATASE_SIZE)


Examples: 
- input: sequence [2,3,4] output: 234
- input: sequence [0,0,4] output: 4


In [1]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
import numpy as np

## Model

In [2]:
model = Sequential()
model.add(LSTM(128,input_shape=(None,1)))
model.add(Dense(1))
model.summary()

model.compile(loss='mean_absolute_error', optimizer="adam",metrics=['mae','mse'])
num_epochs = 0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               66560     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


## Dataset creation

In [3]:
DATASET_SIZE=256

samples = []
labels = []

for i in range(DATASET_SIZE):
    labels.append(i)
    samples.append( list(str(i).rjust(3,'0')) )

samples = np.array(samples,dtype=float)
samples = np.expand_dims(samples,axis=2)
labels = np.array(labels,dtype=int)

print("Sample (input):\n",samples[123])
print("Label (output):",labels[123])
print("Samples size",samples.shape)
print("Labels size",labels.shape)

Sample (input):
 [[1.]
 [2.]
 [3.]]
Label (output): 123
Samples size (256, 3, 1)
Labels size (256,)


In [4]:
TRAINING_SIZE = .5
from sklearn.model_selection import train_test_split
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

Training samples: 128  test samples 128


In [7]:
EPOCHS=100
BATCH_SIZE = int(len(trainSamples)/4)
print('Training with',len(trainSamples),'samples',EPOCHS,'epochs and batch_size=',BATCH_SIZE)
for x in range(10):
    H = model.fit(trainSamples, trainLabels, epochs=EPOCHS,verbose=0,batch_size=BATCH_SIZE)
    num_epochs += EPOCHS
    print()
    print("Final epochs: {} - loss={:6.3f}, loss improvement={:6.3f}".
          format(num_epochs,H.history['loss'][-1], H.history['loss'][0]-H.history['loss'][-1]))
    check_model()
print("Done")

Training with 128 samples 100 epochs and batch_size= 32

Final epochs: 200 - loss=39.692, loss improvement=30.351
Average error: 38.737

Final epochs: 300 - loss=22.331, loss improvement=17.165
Average error: 21.048

Final epochs: 400 - loss=10.828, loss improvement=11.266
Average error: 10.154

Final epochs: 500 - loss= 4.999, loss improvement= 5.712
Average error:  4.859

Final epochs: 600 - loss= 2.626, loss improvement= 2.502
Average error:  2.504

Final epochs: 700 - loss= 1.541, loss improvement= 1.099
Average error:  1.112

Final epochs: 800 - loss= 0.648, loss improvement= 0.850
Average error:  0.666

Final epochs: 900 - loss= 0.402, loss improvement= 0.405
Average error:  0.317

Final epochs: 1000 - loss= 0.430, loss improvement=-0.015
Average error:  0.258

Final epochs: 1100 - loss= 0.225, loss improvement= 0.191
Average error:  0.191
Done


In [8]:
def check_model(verbose=0,show_training=1):
    pred = model.predict(samples)
    for i in range(len(pred)):
        if(not show_training and i in trainLabels): continue
        p = pred[i][0]
        l = labels[i]
        if(verbose==1):
            train=''
            if i in trainLabels: train='[T]'
            print("{:2} > {:6.2f}, error ={:6.2f} {}".format(l,p,abs(p-l),train))
    avg_error = np.mean(np.absolute(pred-np.expand_dims(labels,axis=1)))
    print('Average error: {:6.3f}'.format(avg_error))
    return(avg_error)    
check_model(1)

 0 >   0.22, error =  0.22 
 1 >   1.09, error =  0.09 [T]
 2 >   2.11, error =  0.11 [T]
 3 >   3.17, error =  0.17 [T]
 4 >   4.24, error =  0.24 [T]
 5 >   5.29, error =  0.29 [T]
 6 >   6.33, error =  0.33 
 7 >   7.40, error =  0.40 [T]
 8 >   8.58, error =  0.58 
 9 >   9.91, error =  0.91 
10 >  10.18, error =  0.18 [T]
11 >  11.32, error =  0.32 
12 >  12.32, error =  0.32 [T]
13 >  13.31, error =  0.31 [T]
14 >  14.32, error =  0.32 
15 >  15.32, error =  0.32 [T]
16 >  16.32, error =  0.32 [T]
17 >  17.32, error =  0.32 
18 >  18.32, error =  0.32 [T]
19 >  19.33, error =  0.33 [T]
20 >  20.83, error =  0.83 
21 >  21.48, error =  0.48 
22 >  22.32, error =  0.32 
23 >  23.24, error =  0.24 [T]
24 >  24.19, error =  0.19 
25 >  25.16, error =  0.16 [T]
26 >  26.14, error =  0.14 
27 >  27.15, error =  0.15 
28 >  28.17, error =  0.17 [T]
29 >  29.20, error =  0.20 
30 >  30.09, error =  0.09 [T]
31 >  31.10, error =  0.10 [T]
32 >  32.13, error =  0.13 
33 >  33.16, error =  

0.19100613798946142

In [9]:
model.predict([[[2],[2],[1]]])

array([[220.68341]], dtype=float32)

In [ ]:
model.save('model_135_999.h5')